### Initial

In [9]:
# packages
import sqlite3
import pandas as pd

In [10]:
# load data
df = pd.read_csv("../data/clean_fish_dataset.csv")

In [11]:
# take a peek
df.head()

,SpecCode,Genus,Species,FBname,Length,LTypeMaxM,Weight,BodyShapeI,Fresh,Brack,...,IUCN_Code,Resilience,Vulnerability,Aquarium,GameFish,Importance,UsedforAquaculture,SizeClass,CommericalStatus,ThreatenedStatus
0,64588,Aapticheilichthys,Aapticheilichthys websteri,NaN,3.08,SL,46243.196613,fusiform / normal,1,0,...,NE,Unkwon,10.00,never/rarely,0,Unknown,never/rarely,Small,0,0
1,16239,Aaptosyax,Aaptosyax grypus,Giant salmon carp,130.00,SL,30000.000000,fusiform / normal,1,0,...,CR,Very low,90.00,never/rarely,0,Unknown,never/rarely,Large,0,1
2,2347,Abactochromis,Abactochromis labrosus,NaN,11.50,SL,46243.196613,short and / or deep,1,0,...,LC,High,10.00,commercial,0,commercial,never/rarely,Small,1,0
3,62612,Abalistes,Abalistes filamentosus,NaN,32.50,SL,1375.000000,short and / or deep,0,0,...,LC,Medium,29.65,never/rarely,0,Unknown,never/rarely,Medium,0,0
4,9,Abalistes,Abalistes stellatus,Starry triggerfish,60.00,TL,46243.196613,short and / or deep,0,0,...,LC,Medium,44.00,commercial,0,commercial,never/rarely,Medium,1,0


In [4]:
# connect/create the db file
conn = sqlite3.connect("fish_species_project.db")

In [5]:
# df to db table called fish_species_traits
df.to_sql("fish_species_traits", conn, if_exists = "replace", index = False)

36856

In [6]:
# take a peek at the db
print(pd.read_sql("SELECT * FROM fish_species_traits LIMIT 5;", conn))

   SpecCode              Genus                     Species  \
0     64588  Aapticheilichthys  Aapticheilichthys websteri   
1     16239          Aaptosyax            Aaptosyax grypus   
2      2347      Abactochromis      Abactochromis labrosus   
3     62612          Abalistes      Abalistes filamentosus   
4         9          Abalistes         Abalistes stellatus   

               FBname  Length LTypeMaxM        Weight           BodyShapeI  \
0                None    3.08        SL  46243.196613    fusiform / normal   
1   Giant salmon carp  130.00        SL  30000.000000    fusiform / normal   
2                None   11.50        SL  46243.196613  short and / or deep   
3                None   32.50        SL   1375.000000  short and / or deep   
4  Starry triggerfish   60.00        TL  46243.196613  short and / or deep   

   Fresh  Brack  ...  IUCN_Code Resilience Vulnerability      Aquarium  \
0      1      0  ...         NE     Unkwon         10.00  never/rarely   
1      1  

#### DB is now populated

---

### SQL query and plot

In [8]:
# SQL query
query1 = "SELECT species, weight, length, CommercialStatus FROM fish_species_traits WHERE weight > 250"
# new df based on 250+ grams
df_heavy_fish = pd.read_sql(query1, conn)

DatabaseError: Execution failed on sql 'SELECT species, weight, length, CommercialStatus FROM fish_species_traits WHERE weight > 250': no such column: CommercialStatus

In [ ]:
# plot
df_heavy.plot.scatter(x='length', y='weight')